In [1]:
# Load brain data and mask

# Access nifti, bval & bvec files
from os.path import expanduser, join
home = expanduser('~')
dname = join(home, 'brain-data-neil')

fdwi = join(dname, 'cdmri11_r.nii')
mask = join(dname, 'vol0083_brain_mask.nii.gz')
fbval = join(dname, 'parameters_new_bval.txt')
fbvec = join(dname, 'parameters_new_bvec2.txt')

# Load dMRI datasets 
from dipy.io.image import load_nifti
data, affine, img = load_nifti(fdwi, return_img=True)
mask_data, affine1, img1 = load_nifti(mask, return_img=True)

# Check size of data --> (77, 92, 56, 1344)
print(data.shape)

(77, 92, 56, 1344)


In [2]:
# Set up acquisition scheme using bval, bvec

# Import relevant modules
from dmipy.core import modeling_framework
from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

# Load parameters and convert to SI units
bvalues = np.loadtxt(join('parameters_new_bval.txt'))  # given in s/m^2
bvalues_SI = bvalues * 1e6 # now given in SI units as s/mm^2
gradient_directions = np.loadtxt(join('parameters_new_bvec.txt'))

# delta and Delta times in seconds
delta = 0.0242
Delta = 0.0391

# Acquisition scheme
acq_scheme = acquisition_scheme_from_bvalues(bvalues_SI, gradient_directions, delta, Delta)
acq_scheme.print_acquisition_info

Acquisition scheme summary

total number of measurements: 1344
number of b0 measurements: 84
number of DWI shells: 4

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms] |TE[ms]
0           |84        |0               |0                        |24.2       |39.1      |N/A  
1           |84        |500             |19                       |24.2       |39.1      |N/A  
2           |252       |1000            |27                       |24.2       |39.1      |N/A  
3           |420       |2000            |39                       |24.2       |39.1      |N/A  
4           |504       |3000            |48                       |24.2       |39.1      |N/A  


In [3]:
# Create ball, sphere & stick model

# Import relevant modules
from dmipy.signal_models import gaussian_models, cylinder_models, sphere_models
from dmipy.core.modeling_framework import MultiCompartmentModel

# Create ball, sphere and stick model
ball = gaussian_models.G1Ball()
sphere = sphere_models.S4SphereGaussianPhaseApproximation()
stick = cylinder_models.C1Stick()
bss = MultiCompartmentModel(models=[ball, sphere, stick])

print(bss.parameter_cardinality)

# Define 1 voxel and store in variable E
x_pos = 64
y_pos = 76
z_pos = 34
E = data[x_pos,y_pos,z_pos,:]
# print(E.shape)

# Fit this voxel to the model
fitted_BSS = bss.fit(acq_scheme, E)

OrderedDict([('G1Ball_1_lambda_iso', 1), ('S4SphereGaussianPhaseApproximation_1_diameter', 1), ('C1Stick_1_mu', 2), ('C1Stick_1_lambda_par', 1), ('partial_volume_0', 1), ('partial_volume_1', 1), ('partial_volume_2', 1)])
Using parallel processing with 4 workers.
Setup brute2fine optimizer in 46.580337047576904 seconds
Fitting of 1 voxels complete in 1.9387807846069336 seconds.
Average of 1.9387807846069336 seconds per voxel.


In [4]:
fitted_BSS.fitted_parameters

{'G1Ball_1_lambda_iso': array([3.e-09]),
 'S4SphereGaussianPhaseApproximation_1_diameter': array([1.99792377e-05]),
 'C1Stick_1_mu': array([[1.60862899, 2.49203394]]),
 'C1Stick_1_lambda_par': array([1.e-10]),
 'partial_volume_0': array([0.86113079]),
 'partial_volume_1': array([0.00138869]),
 'partial_volume_2': array([0.13748052])}